In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:2px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:11pt;padding:4px;}
table.dataframe{font-size:10px;}
</style>
"""))

**<font size='6' color='red'>ch5. LSTM/GRU</font>**
- 5만 영화 감상평(독립변수) -> 부정/긍정(타켓변수)
## 1. 패키지

In [2]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from time import time # 70.01.01 부터 현재까지의 밀리세컨

from sklearn.metrics import confusion_matrix,f1_score,precision_score, recall_score

## 2. 하이퍼 파라미터 설정(이 파라미터를 바꾸면 정확도나 학습 속도에 차이남)

In [30]:
MY_WORDS = 10000  # imdb 데이터의 단어수
MY_LENGTH = 80  # 영화평 단어수 80개만 독립변수
MY_EMBED = 32   # embading layer의 결과 차원
MY_HIDDEN = 64  # LSTM의 units 차원
MY_EPOCH = 10  # 학습 수
MY_BATCH = 200 # batch_size(fit시 매번 데이터를 가져오는 데이터)

## 3. 데이터 불러오기

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MY_WORDS)

In [5]:
print('학습셋 입력변수 모양 : ', x_train.shape)
print('학습셋 타겟변수 모양 : ', y_train.shape)
print('학습셋 샘플 : ', len(x_train[1]))
print('학습셋 샘플 : ', type(x_train[1]), x_train[1], y_train[1])
print('테스트셋 변수들 모양 : ', x_test.shape, y_test.shape)

학습셋 입력변수 모양 :  (25000,)
학습셋 타겟변수 모양 :  (25000,)
학습셋 샘플 :  189
학습셋 샘플 :  <class 'list'> [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95] 0
테스트셋 변수들 모양 :  (25000,) (25000,)


In [6]:
# 긍정/부정 갯수
print('학습셋의 긍정 갯수 : ', y_train.sum())
print('테스트셋의 긍정 갯수 : ', y_test.sum())

학습셋의 긍정 갯수 :  12500
테스트셋의 긍정 갯수 :  12500


## 4. 문자단어 -> 정수

In [7]:
word_to_id = imdb.get_word_index()  # {'word':id}
print(word_to_id['movie'])
print(word_to_id['film'])
print(word_to_id['sonja'])
print(word_to_id['a'])
print(word_to_id['the'])
# 정수 -> 문자 단어
id_to_word = {}  # {1:'the', 3:'a', 16816:'sonja'}
for word, value in word_to_id.items():
    id_to_word[value] = word
print(id_to_word[1])
print(id_to_word[3])
print(id_to_word[2])

17
19
16816
3
1
the
a
and


In [8]:
msg = 'What a wonderful movie'
msg = msg.lower().split()
# 1: 리뷰시작을 알리는 숫자, 2: 문자가 짤려서 잘 못 읽어옴, 3: padding처리
data = [1] + [word_to_id.get(m, -1)+3 for m in msg]
print('원 후기 내용 : ', msg)
print('encoded된 data : ', data)
print('추정된 data : ', [id_to_word.get(b-3, '???') for b in data])
print('추정된 data : ', ' '.join([id_to_word.get(b-3, '???') for b in data]))

원 후기 내용 :  ['what', 'a', 'wonderful', 'movie']
encoded된 data :  [1, 51, 6, 389, 20]
추정된 data :  ['???', 'what', 'a', 'wonderful', 'movie']
추정된 data :  ??? what a wonderful movie


## 5.숫자 영화평 -> 자연어 영화평 return 함수

In [9]:
def decoding(review_num):
    decoded = [id_to_word.get(num-3, '???') for num in review_num]
    return ' '.join(decoded)

In [10]:
print(decoding(x_train[1]), y_train[1])

??? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ??? the hair is big lots of boobs ??? men wear those cut ??? shirts that show off their ??? sickening that men actually wore them and the music is just ??? trash that plays over and over again in almost every scene there is trashy music boobs and ??? taking away bodies and the gym still doesn't close for ??? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then 0


## 6. 영화평(입력변수)의 길이

In [11]:
def show_length(x_train):
    print('첫 20개 영화평 길이')
    print([len(x_data) for x_data in x_train[:20]])

In [12]:
# pad_sequence 전
show_length(x_train)

첫 20개 영화평 길이
[218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 450, 99, 117, 238, 109, 129, 163, 752, 212, 177]


In [13]:
print('제일 긴 영화평 단어 길이 : ', 
     max(len(x_data) for x_data in x_train))
print('제일 짧은 영화평 단어 길이 : ', 
     min(len(x_data) for x_data in x_train))
print('영화평 단어 길이 중위수 : ',
     np.median([len(x_data) for x_data in x_train]))

제일 긴 영화평 단어 길이 :  2494
제일 짧은 영화평 단어 길이 :  11
영화평 단어 길이 중위수 :  178.0


## 7. 모든 영화평 길이를 동일하게 (80)

In [14]:
X_train = pad_sequences(x_train,
                       padding='pre',
                       truncating='pre',  # 뒷부분을 짜르고 앞 부분을 남김 / 'pre' : 앞 부분을 짜르고 뒷 부분을 남김
                       maxlen=MY_LENGTH)
X_test = pad_sequences(x_test,
                      padding='pre',
                      truncating='pre',
                      maxlen=MY_LENGTH)
show_length(X_train), show_length(X_test)

첫 20개 영화평 길이
[80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80]
첫 20개 영화평 길이
[80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80]


(None, None)

## 8. 최종 데이터 shape확인

In [15]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 80), (25000,), (25000, 80), (25000,))

In [16]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [17]:
X_train[0]

array([  15,  256,    4,    2,    7, 3766,    5,  723,   36,   71,   43,
        530,  476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,
        104,   88,    4,  381,   15,  297,   98,   32, 2071,   56,   26,
        141,    6,  194, 7486,   18,    4,  226,   22,   21,  134,  476,
         26,  480,    5,  144,   30, 5535,   18,   51,   36,   28,  224,
         92,   25,  104,    4,  226,   65,   16,   38, 1334,   88,   12,
         16,  283,    5,   16, 4472,  113,  103,   32,   15,   16, 5345,
         19,  178,   32])

## 9. 모델  생성

In [18]:
model = Sequential()
model.add(Embedding(input_dim=MY_WORDS,  # 10000
                   output_dim=MY_EMBED,  # 32
                   input_length=MY_LENGTH,  # 80
                   ))
# RNN : 입력 단어의 길이 수가 너무 길면 파라미터 업데이트 안 됨
# 개선모델 1. LSTM | 개선모델 2. GRU
model.add(LSTM(units=MY_HIDDEN,
              input_shape=(MY_LENGTH, MY_EMBED)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_________________________________________________________________


## 10. 학습환경 설정 및 학습하기

In [19]:
model.compile(loss='binary_crossentropy',  # 이진분류시 손실함수
             optimizer='adam',
             metrics=['acc'])
begin = time() # 70.1.1부터 현재까지의 세컨
print(begin)
hist = model.fit(X_train, y_train,
                epochs=MY_EPOCH,
                batch_size=MY_BATCH,
                validation_split=0.2,
                verbose=1)
end = time()
print('총 학습 시간 : ', (end-begin))

In [20]:
hist.history.keys()

NameError: name 'hist' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig, loss_ax = plt.subplots(figsize=(12,6))
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
acc_ax = loss_ax.twinx()
acc_ax.plot(hist.history['acc'], 'g', label='train accuracy')
acc_ax.plot(hist.history['val_acc'], 'b', label='val accuracy')
loss_ax.set_xlabel('epochs')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')
loss_ax.legend(loc='center right')
acc_ax.legend(loc='upper left')
plt.show()

## 11. 모델 평가

In [ ]:
# pad_sequences 'post' : 75.6%
# pad_sequences 'pre' : 80.1%
loss, acc = model.evaluate(X_test, y_test)
print('정확도 : ', acc)

In [ ]:
# 혼동행렬, recall, precision을 위한 yhat
yhat = (model.predict(X_test, verbose=0) > 0.5).astype(np.int16).reshape(-1)
yhat

In [ ]:
# 혼동행렬
confusion_matrix(y_test, yhat)

In [ ]:
# recall ( 실제 True인 것 중 True로 예측한 비율) 9853 /  ( 2647+9853)
recall_score(y_test, yhat)

In [ ]:
# precision(True로 예측한 것 중 실제값이 True인 비율) 9853 / (2307+9853)
precision_score(y_test, yhat)

## 12. 모델 사용하기

In [39]:
review = """This was genuinely one of those rare films where I completely lost track of time in the theater. 
Initially, I expected it to follow a predictable storyline, 
but from the midpoint onward, the plot kept subverting my expectations, keeping me thoroughly engaged until the very end.
The performances by the lead actors were particularly impressive. 
They managed to convey even the most subtle emotional nuances with remarkable naturalness. 
The cinematography and soundtrack also complemented the story perfectly, significantly enhancing the overall immersive experience.
What I appreciated most was how the film lingered in my thoughts long after leaving the theater. 
It sparked plenty of conversations with friends, and I found myself wanting to watch it again – that's how satisfying the experience was.
It's the kind of movie that works both as light entertainment and as something that gives you plenty to think about. 
I'd definitely recommend it to a wide audience!~!@#$%^&
""".lower()
import re
review = re.sub('[^a-zA-Z\'\s]', '', review)
#print('영화평(특수문자 제외) : ', review)
review = review.split()  # 단어 list
review = [1] + [word_to_id.get(word, -1)+3 if word_to_id.get(word, -1) < 10000 else '???' for word in review]
print(review, len(review))

[1, 14, 16, 2070, 31, 7, 148, 1281, 108, 121, 13, 340, 416, 1406, 7, 58, 11, 4, 750, 2721, 13, 873, 12, 8, 794, 6, 727, 769, 21, 39, 4, '???', '???', 4, 114, 828, '???', 61, 1398, 1895, 72, 1562, 3953, 366, 4, 55, 130, 4, 354, 34, 4, 485, 156, 71, 572, 1159, 36, 1319, 8, 2833, 60, 4, 91, 1302, 921, 6902, 19, 1739, '???', 4, 627, 5, 816, 82, '???', 4, 65, 950, 8703, '???', 4, 444, '???', 585, 51, 13, 2525, 91, 16, 89, 4, 22, '???', 11, 61, 2334, 196, 103, 1200, 4, 750, 12, '???', 958, 7, 3958, 19, 369, 5, 13, 258, 546, 1786, 8, 106, 12, 174, 198, 89, 2349, 4, 585, 16, 45, 4, 243, 7, 20, 15, 495, 199, 17, 641, 722, 5, 17, 142, 15, 408, 25, 958, 8, 104, 44, 474, 407, 386, 12, 8, 6, 1876, 311] 152


In [40]:
input_data = pad_sequences([review],
                      padding='pre',
                      truncating='pre',
                      maxlen=MY_LENGTH)
input_data

ValueError: invalid literal for int() with base 10: '???'

In [41]:
result = (model.predict(input_data) > 0.5).astype('int8').reshape(-1)
result

1/1 [==============================] - 0s 25ms/step


array([0], dtype=int8)

In [36]:
review = """The movie was really exciting, 
and I watched it throughout the movie hoping it wouldn't end without 
watching the clock. I strongly recommend it. 
It was even more interesting thinking that it could happen in real life. 
The main character was handsome and acted well, so my eyes were happy, 
and the story was fun @_@ㅠ.ㅠ""".lower()
import re
review = re.sub('[^a-zA-Z\'\s]', '', review)
review = review.split() # 단어 list
review = [1] + [word_to_id.get(word, -1)+3 for word in review]
print(review, len(review))

[1, 4, 20, 16, 66, 1127, 5, 13, 296, 12, 469, 4, 20, 1383, 12, 586, 130, 209, 149, 4, 5431, 13, 2303, 386, 12, 12, 16, 60, 53, 221, 536, 15, 12, 100, 593, 11, 147, 113, 4, 293, 109, 16, 2252, 5, 917, 73, 38, 61, 523, 71, 654, 5, 4, 65, 16, 253] 56


In [37]:
input_data = pad_sequences([review],
                      padding='pre',
                      truncating='pre',
                      maxlen=MY_LENGTH)
input_data

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    4,   20,   16,   66, 1127,    5,   13,  296,
          12,  469,    4,   20, 1383,   12,  586,  130,  209,  149,    4,
        5431,   13, 2303,  386,   12,   12,   16,   60,   53,  221,  536,
          15,   12,  100,  593,   11,  147,  113,    4,  293,  109,   16,
        2252,    5,  917,   73,   38,   61,  523,   71,  654,    5,    4,
          65,   16,  253]])

In [38]:
result = (model.predict(input_data)>0.5).astype('int8').reshape(-1)
result

1/1 [==============================] - 0s 63ms/step


array([0], dtype=int8)